In [1]:
# load modules and set configurations
import numpy as np
import pandas as pd

import os, copy, random, pickle, gc
from itertools import product
from tqdm import tqdm

pd.set_option('display.max_columns', None)

import torch

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED = 42

In [2]:
with open(f'df_std_hos_dict_6hour.pkl', 'rb') as f:
    df_raw_icu_dict = pickle.load(f)

In [3]:
for i in df_raw_icu_dict.keys():
    df_raw_icu_dict[i] = {
        'hadm_id': df_raw_icu_dict[i]['hadm_id'].to_numpy().reshape(-1,83,1),
        'X': df_raw_icu_dict[i].drop(labels=['hadm_id', 'time'], axis=1).to_numpy().reshape(-1,83,383)}

In [4]:
import torch
from torch import nn, optim
import torch.nn.functional as F 
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
torch.set_default_dtype(torch.float32)

data_loaders = {}

for i in tqdm(df_raw_icu_dict.keys()):
    tmp_X = torch.tensor(df_raw_icu_dict[i]['X'].astype(np.float32))
    tmp_ids = torch.tensor(df_raw_icu_dict[i]['hadm_id'].astype(int))
    if i in ['train_x', 'val_x']:
        data_loaders[i] = DataLoader(dataset=TensorDataset(tmp_X, tmp_ids), batch_size=128, shuffle=False)
    else:
        data_loaders[i] = DataLoader(dataset=TensorDataset(tmp_X, tmp_ids), batch_size=1, shuffle=False)

100%|██████████| 4/4 [00:47<00:00, 11.99s/it]


In [5]:
class Encoder(nn.Module):
  def __init__(self, seq_len, n_features, embedding_dim):
    super(Encoder, self).__init__()
    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim = embedding_dim
    self.hidden_dim =  2 * embedding_dim
    self.rnn1 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )

  def forward(self, x):
    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)
    return hidden_n.reshape((-1,1, self.embedding_dim))


class Decoder(nn.Module):
  def __init__(self, seq_len, input_dim, n_features=383):
    super(Decoder, self).__init__()
    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim = 2 * input_dim
    self.n_features = n_features
    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.output_layer = nn.Linear(self.hidden_dim, n_features)

  def forward(self, x):
    x = x.repeat(1,self.seq_len, 1)
    x, (hidden_n, cell_n) = self.rnn1(x)
    return self.output_layer(x)

In [6]:
class RecurrentAutoencoder(nn.Module):
  def __init__(self, seq_len, n_features, embedding_dim):
    super(RecurrentAutoencoder, self).__init__()
    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

    return x

In [32]:
# training setting
seed_everything(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hidden_unit = 512 # 64, 128, 256, 512
n_epochs, factor, patience, min_lr = (3000, 0.1, 100, 1e-6)
loss_reduction = 'stay_wise_mean'
early_patience=200
gc.collect()
torch.cuda.empty_cache()
model = RecurrentAutoencoder(83, 383, hidden_unit)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=factor, patience=patience, min_lr=min_lr, verbose=True)
criterion = nn.MSELoss(reduction='none').to(device)

history = dict(train=[], val=[])

best_model_wts = copy.deepcopy(model.state_dict())
best_loss = float('inf')
early_stopping_counter = 0

In [33]:
data='std' #raw, std, norm

In [34]:
# training
for epoch in range(1, n_epochs+1):
    model = model.train()

    train_losses = []
    for seq_true, _ in data_loaders['train_x']:
        seq_true = seq_true.to(device)
        mask = ~torch.all(seq_true==0, axis=2)
        seq_pred = model(seq_true)

        if loss_reduction == 'stay_wise_mean':
            l = criterion(seq_pred[mask], seq_true[mask]).sum(axis=1)
            lens = mask.sum(axis=1).detach().cpu()
            c_lens = lens.cumsum(dim=0)
            loss = 0
            for idx, i in enumerate(c_lens):
                s = 0 if idx == 0 else c_lens[idx-1]
                loss += l[s:i].sum()/lens[idx]
        elif loss_reduction == 'global_mean':
            l = criterion(seq_pred[mask], seq_true[mask]).sum()
            loss = l/mask.sum()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        #print(len(train_losses))

    val_losses = []
    model = model.eval()
    with torch.no_grad():
        for seq_true, _ in data_loaders['val_x']:
            seq_true = seq_true.to(device)
            mask = ~torch.all(seq_true==0, axis=2)
            seq_pred = model(seq_true)

            if loss_reduction == 'stay_wise_mean':
                l = criterion(seq_pred[mask], seq_true[mask]).sum(axis=1)
                lens = mask.sum(axis=1).detach().cpu()
                c_lens = lens.cumsum(dim=0)
                loss = 0
                for idx, i in enumerate(c_lens):
                    s = 0 if idx == 0 else c_lens[idx-1]
                    loss += l[s:i].sum()/lens[idx]
            elif loss_reduction == 'global_mean':
                l = criterion(seq_pred[mask], seq_true[mask]).sum()
                loss = l/mask.sum()

            val_losses.append(loss.item())

    train_loss = np.mean(train_losses)
    #print(len(train_losses))
    val_loss = np.mean(val_losses)

    history['train'].append(train_loss)
    history['val'].append(val_loss)

    print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')

    scheduler.step(val_loss)

    print("Current learning rate:", optimizer.param_groups[0]['lr'])

    if val_loss < best_loss:
        best_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_patience:
            print(f'Early stopping at epoch {epoch} due to no improvement in validation loss.')
            break
    
    print(f'early_stopping_counter: {early_stopping_counter} ')
    

model.load_state_dict(best_model_wts)
torch.save(model.state_dict(), f'best_moel-{data}-hos-{loss_reduction}.pth')


Epoch 1: train loss 18819.182555786865 val loss 21744.560444078947
Current learning rate: 0.001
early_stopping_counter: 0 
Epoch 2: train loss 17085.014364094077 val loss 17506.47583650288
Current learning rate: 0.001
early_stopping_counter: 0 
Epoch 3: train loss 12504.412716071763 val loss 15120.38655491879
Current learning rate: 0.001
early_stopping_counter: 0 
Epoch 4: train loss 10842.952055240106 val loss 13735.885604055304
Current learning rate: 0.001
early_stopping_counter: 0 
Epoch 5: train loss 9628.445900922763 val loss 13034.112458881578
Current learning rate: 0.001
early_stopping_counter: 0 
Epoch 6: train loss 8665.005637528653 val loss 12245.87826538086
Current learning rate: 0.001
early_stopping_counter: 0 
Epoch 7: train loss 7760.64486200938 val loss 11308.74271432977
Current learning rate: 0.001
early_stopping_counter: 0 
Epoch 8: train loss 7052.687635958551 val loss 10784.972810444078
Current learning rate: 0.001
early_stopping_counter: 0 
Epoch 9: train loss 6605.

In [37]:
model

RecurrentAutoencoder(
  (encoder): Encoder(
    (rnn1): LSTM(383, 1024, batch_first=True)
    (rnn2): LSTM(1024, 512, batch_first=True)
  )
  (decoder): Decoder(
    (rnn1): LSTM(512, 1024, batch_first=True)
    (output_layer): Linear(in_features=1024, out_features=383, bias=True)
  )
)

In [38]:
# evaluation data
gc.collect()
torch.cuda.empty_cache()

seed_everything(SEED)
hidden_unit = 256

model = RecurrentAutoencoder(83, 383, 512)
model = model.to(device)
model.load_state_dict(torch.load(f'best_moel-std-hos-global_mean.pth'))
model = model.eval()

criterion = nn.MSELoss(reduction='mean').to(device)

# loss calculation
eval_split = 'val_test_x' #tst val_th

eval_data = []
with torch.no_grad():
    for seq_true, stay_id in tqdm(data_loaders[eval_split]):
        s_id = stay_id.cpu().numpy().ravel()[0]
        seq_true = seq_true.to(device)
        seq_pred = model(seq_true)
        loss=criterion(seq_pred, seq_true)
        
        eval_data.append([s_id,loss.item()])

eval_data = pd.DataFrame(eval_data, columns=['hadm_id', 'score'])


100%|██████████| 13585/13585 [01:51<00:00, 121.73it/s]


In [39]:
hos_data=pd.read_csv('hos_label.csv')

In [40]:
i_d=hos_data[hos_data['hadm_id'].isin(eval_data['hadm_id'])]
eval_data_1=pd.merge(eval_data,i_d,on='hadm_id',how='left')
eval_data_1

,hadm_id,score,label
0,20004038,0.533074,0
1,20004038,0.515734,0
2,20004038,0.596503,0
3,20004038,0.595708,0
4,20004038,0.487797,0
...,...,...,...
13580,29996041,0.550587,0
13581,29996041,0.537918,0
13582,29996041,0.533166,0
13583,29996041,0.539521,0


In [41]:
eval_split='val_test_x'
eval_data_1.to_csv(f"eval_{data}-data-hos-{eval_split}-{loss_reduction}.csv", index=False)

In [42]:
# evaluation data
gc.collect()
torch.cuda.empty_cache()

seed_everything(SEED)
hidden_unit = 512

model = RecurrentAutoencoder(83, 383, hidden_unit)
model = model.to(device)
model.load_state_dict(torch.load(f'best_moel-{data}-hos-{loss_reduction}.pth'))
model = model.eval()

criterion = nn.MSELoss(reduction='mean').to(device)

# loss calculation
eval_split = 'test_x' #tst val_th

eval_data_2 = []
with torch.no_grad():
    for seq_true, hadm_id in tqdm(data_loaders[eval_split]):
        s_id = hadm_id.cpu().numpy().ravel()[0]
        seq_true = seq_true.to(device)
        seq_pred = model(seq_true)
        loss=criterion(seq_pred, seq_true)
        
        eval_data_2.append([s_id,loss.item()])

eval_data_2 = pd.DataFrame(eval_data_2, columns=['hadm_id', 'score'])

100%|██████████| 14340/14340 [02:00<00:00, 118.91it/s]


In [43]:
i_d2=hos_data[hos_data['hadm_id'].isin(eval_data_2['hadm_id'])]
eval_data_2=pd.merge(eval_data_2,i_d2,on='hadm_id',how='left')
eval_data_2

,hadm_id,score,label
0,20022720,0.463378,0
1,20022720,0.492806,0
2,20022720,0.481417,0
3,20022720,0.466180,0
4,20022720,0.461356,0
...,...,...,...
14335,29997500,0.394761,1
14336,29997500,0.390293,1
14337,29997500,0.384276,1
14338,29997500,0.376598,1


In [44]:
eval_split='test_x_6_hour'
eval_data_2.to_csv(f"eval_{data}-data-hos-{eval_split}-{loss_reduction}.csv", index=False)

In [45]:
def conf_mat(true, pred):
    tp = ((pred == 1) & (true == 1)).sum()
    fp = ((pred == 1) & (true == 0)).sum()
    fn = ((pred == 0) & (true == 1)).sum()
    tn = ((pred == 0) & (true == 0)).sum()
    return tp, fp, fn, tn

eval_result = []

scores = eval_data_1['score'].unique()

for s in tqdm(scores):
    eval_data_1['pred'] = np.where(eval_data_1['score']>=s, 1, 0)
    tmp = eval_data_1.groupby('hadm_id').agg({'label': lambda x: x.values[0], 'pred': 'max'}).reset_index()
    tp, fp, fn, tn = conf_mat(tmp['label'], tmp['pred'])

    eval_result.append([s, tp/(tp+fn), tp/(tp+fp), 2*tp/(fp+2*tp+fn)])

eval_result = pd.DataFrame(eval_result, columns=['score', 'rec', 'prec', 'f1'])


100%|██████████| 13567/13567 [03:06<00:00, 72.83it/s]


In [46]:
eval_split='val_test_x_6_hour'
eval_result.to_csv(f"eval_reult_{data}-data-hos-{eval_split}-{loss_reduction}.csv", index=False)

In [48]:
eval_result

,score,rec,prec,f1
0,0.533074,0.950431,0.507480,0.661665
1,0.515734,0.987069,0.498368,0.662328
2,0.596503,0.607759,0.692875,0.647532
3,0.595708,0.607759,0.692875,0.647532
4,0.487797,1.000000,0.500000,0.666667
...,...,...,...,...
13562,0.550587,0.797414,0.558069,0.656610
13563,0.537918,0.905172,0.520446,0.660897
13564,0.533166,0.950431,0.508065,0.662162
13565,0.539521,0.887931,0.526854,0.661316


In [47]:
eval_result[eval_result['f1']>=eval_result['f1'].max()]

,score,rec,prec,f1
4,0.487797,1.0,0.5,0.666667
5,0.486028,1.0,0.5,0.666667
6,0.442199,1.0,0.5,0.666667
7,0.455245,1.0,0.5,0.666667
8,0.427778,1.0,0.5,0.666667
...,...,...,...,...
13525,0.468516,1.0,0.5,0.666667
13526,0.467816,1.0,0.5,0.666667
13527,0.463314,1.0,0.5,0.666667
13528,0.452889,1.0,0.5,0.666667


In [54]:
def conf_mat(true, pred):
    tp = ((pred == 1) & (true == 1)).sum()
    fp = ((pred == 1) & (true == 0)).sum()
    fn = ((pred == 0) & (true == 1)).sum()
    tn = ((pred == 0) & (true == 0)).sum()
    return tp, fp, fn, tn

eval_result = []

scores = eval_data_1['score'].unique()

for s in tqdm(scores):
    eval_data_1['pred'] = np.where(eval_data_1['score']>=s, 1, 0)
    tmp = eval_data_1.groupby('hadm_id').agg(label=('label', lambda x: x.values[0]), alarm_num=('pred','sum')).reset_index()
    tmp['pred'] = np.where(tmp['alarm_num'] >= 5, 1, 0)
    tp, fp, fn, tn = conf_mat(tmp['label'], tmp['pred'])

    eval_result.append([s, tp/(tp+fn), tp/(tp+fp), 2*tp/(fp+2*tp+fn)])

eval_result = pd.DataFrame(eval_result, columns=['score', 'rec', 'prec', 'f1'])


 54%|█████▍    | 7324/13567 [02:43<02:22, 43.67it/s]C:\Users\DAHS\AppData\Local\Temp\ipykernel_21520\2314707552.py:18: RuntimeWarning: invalid value encountered in scalar divide
  eval_result.append([s, tp/(tp+fn), tp/(tp+fp), 2*tp/(fp+2*tp+fn)])
 54%|█████▍    | 7329/13567 [02:43<02:24, 43.04it/s]C:\Users\DAHS\AppData\Local\Temp\ipykernel_21520\2314707552.py:18: RuntimeWarning: invalid value encountered in scalar divide
  eval_result.append([s, tp/(tp+fn), tp/(tp+fp), 2*tp/(fp+2*tp+fn)])
C:\Users\DAHS\AppData\Local\Temp\ipykernel_21520\2314707552.py:18: RuntimeWarning: invalid value encountered in scalar divide
  eval_result.append([s, tp/(tp+fn), tp/(tp+fp), 2*tp/(fp+2*tp+fn)])
 54%|█████▍    | 7335/13567 [02:43<02:14, 46.33it/s]C:\Users\DAHS\AppData\Local\Temp\ipykernel_21520\2314707552.py:18: RuntimeWarning: invalid value encountered in scalar divide
  eval_result.append([s, tp/(tp+fn), tp/(tp+fp), 2*tp/(fp+2*tp+fn)])
100%|██████████| 13567/13567 [05:08<00:00, 43.91it/s]


In [53]:
eval_result[eval_result['f1']>=eval_result['f1'].max()]

,score,rec,prec,f1
1580,0.443247,0.469828,0.436874,0.452752
1625,0.443305,0.469828,0.436874,0.452752
2859,0.443317,0.469828,0.436874,0.452752
5620,0.443278,0.469828,0.436874,0.452752
8375,0.443301,0.469828,0.436874,0.452752
9186,0.443391,0.469828,0.436874,0.452752
9759,0.443427,0.469828,0.436874,0.452752
9927,0.443427,0.469828,0.436874,0.452752
10103,0.443249,0.469828,0.436874,0.452752
10945,0.443329,0.469828,0.436874,0.452752


In [50]:
def conf_mat(true, pred):
    tp = ((pred == 1) & (true == 1)).sum()
    fp = ((pred == 1) & (true == 0)).sum()
    fn = ((pred == 0) & (true == 1)).sum()
    tn = ((pred == 0) & (true == 0)).sum()
    return tp, fp, fn, tn

eval_result = []

#scores = eval_data_1['score'].unique()

for s in tqdm(scores):
    eval_data_1['pred'] = np.where(eval_data_1['score']>=s, 1, 0)
    eval_data_1['pred_shifted'] = eval_data_1.groupby('hadm_id')['pred'].shift(1).fillna(0)    
    eval_data_1['continuous_ones'] = ((eval_data_1['pred'] + eval_data_1['pred_shifted']) == 2).astype(int)
    tmp = eval_data_1.groupby('hadm_id').agg({'label': lambda x: x.values[0], 'continuous_ones': 'max'}).reset_index()
    tp, fp, fn, tn = conf_mat(tmp['label'], tmp['continuous_ones'])

    eval_result.append([s, tp/(tp+fn), tp/(tp+fp), 2*tp/(fp+2*tp+fn)])

eval_result = pd.DataFrame(eval_result, columns=['score', 'rec', 'prec', 'f1'])


 54%|█████▍    | 7327/13567 [01:47<01:29, 69.43it/s]C:\Users\DAHS\AppData\Local\Temp\ipykernel_21520\809627442.py:19: RuntimeWarning: invalid value encountered in scalar divide
  eval_result.append([s, tp/(tp+fn), tp/(tp+fp), 2*tp/(fp+2*tp+fn)])
100%|██████████| 13567/13567 [03:18<00:00, 68.29it/s]


In [51]:
eval_result[eval_result['f1']>=eval_result['f1'].max()]

,score,rec,prec,f1
350,0.502423,0.948276,0.493827,0.649446
952,0.502178,0.948276,0.493827,0.649446
1569,0.502147,0.948276,0.493827,0.649446
2095,0.502420,0.948276,0.493827,0.649446
2609,0.502433,0.948276,0.493827,0.649446
3595,0.502097,0.948276,0.493827,0.649446
4080,0.502475,0.948276,0.493827,0.649446
4704,0.502347,0.948276,0.493827,0.649446
4740,0.502068,0.948276,0.493827,0.649446
6634,0.502095,0.948276,0.493827,0.649446
